In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="cifar10_cnn_local")

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import os

batch_size = 32
num_classes = 10
epochs = 5
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'outputs')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.


In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [5]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Save model and weights with checkpoint callback
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint_cb = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
import sys
sys.path.append(os.path.join(os.getcwd(), 'code'))

from keras_azure_ml_cb import AzureMlKerasCallback

In [7]:
# Create a run
with exp.start_logging(snapshot_directory='.') as run:

    # Create an Azure Machine Learning monitor callback
    azureml_cb = AzureMlKerasCallback(run)

    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True,
            callbacks=[azureml_cb, checkpoint_cb])

    # Load the best model
    model = load_model(model_path)

    # Score trained model
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    run.log('Test loss', scores[0])
    print('Test accuracy:', scores[1])
    run.log('Test accuracy', scores[1])

    # Upload the best model
    run.upload_file(model_name, model_path)

    # Register the best model
    run.register_model(model_name, model_path=model_name, model_framework='TfKeras')

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 37s 739us/step - loss: 2.0104 - accuracy: 0.2611 - val_loss: 1.7679 - val_accuracy: 0.4010
Epoch 2/5
50000/50000 [==============================] - 33s 659us/step - loss: 1.7250 - accuracy: 0.3736 - val_loss: 1.5813 - val_accuracy: 0.4470
Epoch 3/5
50000/50000 [==============================] - 33s 660us/step - loss: 1.5994 - accuracy: 0.4199 - val_loss: 1.4700 - val_accuracy: 0.4837
Epoch 4/5
50000/50000 [==============================] - 33s 655us/step - loss: 1.5167 - accuracy: 0.4523 - val_loss: 1.4154 - val_accuracy: 0.4992
Epoch 5/5
10000/10000 [==============================] - 2s 184us/step
Test loss: 1.3469151679992675
Test accuracy: 0.5307999849319458


WARNING - Tfkeras will be deprecated soon. Use Model.Framework.TENSORFLOW instead.
